In [2]:
#  데이터 처리 모듈
import pandas as pd
# 행렬 등 통계 연산 모듈
import numpy as np
# 지수형 표기법 e를 연속형 변환
pd.options.display.float_format = '{:.4f}'.format
# 타입 어노테이션(Any, Sequence 등의 메서드 활용)
from typing import *

# 구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")
import os

from itertools import permutations
from itertools import combinations

Mounted at /content/drive


### 1.한국신용데이터(KCD) 서울특별시 소상공인(SOHO)의 신용거래정보 추출 데이터를 활용한 파생변수 생성

* **credit_data_list**
  
  2022년 1월 ~ 2023년 3월 15개월의 서울특별시 소상공인 상권별 업종별 신용거래 및 사업장정보 데이터프레임을 담은 리스트 반환

* **변동계수(coefficient of variation, CV)**
  
  표준편차를 산술평균을 기준으로 표준화(standardization)한 것으로, 표준편차를 산술평균으로 나눈 값

  같은 단위를 가지는 표준편차를 평균으로 나누면, 단위가 사라지고 표준화된 수치를 비교

In [193]:
class MergeData():

  def __init__(self, soho_credit_data = None):
    self.soho_credit_data = soho_credit_data

  def mergeSohoCredit(self, date_time1: int, date_time2: int,
                      file_path: str, file_name: str,
                      column_num:int, column_name_list: List):
    # date_time에 202201 ~ 202212, 202301 ~ 202303을 입력값으로 받고,
    # file_path에 해당 데이터가 있는 드라이브 경로 설정
    # file_path = /content/drive/MyDrive/2023BigContest/Archive/"

    credit_data_list: List = []
      # 월별 데이터프레임을 담을 리스트 생성
    date_time_list = [str(date_time1 + n) for n in range(0, 12, 1)]
    date_time_list.extend([str(date_time2 + n) for n in range(0, 3, 1)])

    for date in date_time_list:
      data = pd.read_csv(
          os.path.join(file_path + date + file_name),
          index_col = 0, sep = ",",
          names = column_name_list)
          # header = None)
      data.drop("index", axis = 0, inplace = True)
      data[column_name_list[column_num:]] = data[column_name_list[column_num:]].astype("float")

      # data["손익분기점_매출액_STD"] = data["월임대료"] / (1 - (data["부가가치세_STD"] + data["매입액"]) / V["매출액"]) = 고정비용 / (1-변동비율)
      # data["객단가_STD"] = data["매출액_STD"] / data["고객수_STD"]
      # data["신규고객단가_STD"] = data["매출액_STD"] / data["신규고객수_STD"]
      # data["전체대비_주말매출액비중_STD"] = data["주말매출액_STD"] / data["매출액_STD"]
        # 병합 후 파생컬럼 생성할 것

      def mutateUniqueID(column_상권명, column_업종중분류):
        상권_식별자 = column_상권명
        if not pd.isna(column_업종중분류):
          상권_식별자 += "_" + str(column_업종중분류)
        return 상권_식별자
      data["상권_식별자"] = data.apply(
          lambda x: mutateUniqueID(x["상권명"], x["외식업종_중분류"]), axis = 1)

      credit_data_list.append(data)
        # os.path.join을 통한 구글 드라이브 파일 내 경로로 csv데이터 로드하여 판다스 데이터프레임으로 저장 후
        # credit_data_list에 0 ~ 11번째 인덱스의 원소로 추가

    return credit_data_list
      # 2022년 1월 ~ 2023년 3월 15개월의 서울특별시 소상공인 상권별 업종별 신용거래 및 사업장정보 데이터프레임을 담은 리스트 반환
      # 병합하여 self.soho_credit_data라는 멤버변수에 저장할 것

In [194]:
Preprocess = MergeData()

---

#### [SOHO_INCOME_DATA_LIST] 소호 신용거래정보 매출액(카드 + 배달), 매입액(카드 + 현금), 객단가, 부가가치세, 손익분기점 등 파생변수 생성

In [195]:
SOHO_INCOME_DATA_LIST = Preprocess.mergeSohoCredit(
    date_time1 = 202201,
    date_time2 = 202301,
    file_path = "/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/SOHO_INCOME/SOHO_",
    file_name = "_INCOME_230916.csv",
    column_name_list = ["상권명", "상권코드", "외식업종_중분류",
                        "매출액_표준편차", "매입액_표준편차", "매출총이익_표준편차", "부가가치세_표준편차", "부가세차감전영업이익_표준편차", "영업이익_표준편차",
                        "주말매출액_표준편차", "사업장방문고객수_표준편차", "사업장방문신규고객수_표준편차",
                        "배달매출액_표준편차", "주말배달매출액_표준편차", "카드매출액_표준편차", "주말카드매출액_표준편차"],
    column_num = 3)

In [196]:
SOHO_INCOME_DATA_LIST[0].info()
SOHO_INCOME_DATA_LIST[14]

<class 'pandas.core.frame.DataFrame'>
Index: 170 entries, 0 to 169
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   상권명              170 non-null    object 
 1   상권코드             170 non-null    object 
 2   외식업종_중분류         170 non-null    object 
 3   매출액_표준편차         170 non-null    float64
 4   매입액_표준편차         170 non-null    float64
 5   매출총이익_표준편차       170 non-null    float64
 6   부가가치세_표준편차       170 non-null    float64
 7   부가세차감전영업이익_표준편차  170 non-null    float64
 8   영업이익_표준편차        170 non-null    float64
 9   주말매출액_표준편차       170 non-null    float64
 10  사업장방문고객수_표준편차    170 non-null    float64
 11  사업장방문신규고객수_표준편차  170 non-null    float64
 12  배달매출액_표준편차       154 non-null    float64
 13  주말배달매출액_표준편차     155 non-null    float64
 14  카드매출액_표준편차       170 non-null    float64
 15  주말카드매출액_표준편차     170 non-null    float64
 16  상권_식별자           170 non-null    object 
dtypes: float64(13), objec

,상권명,상권코드,외식업종_중분류,매출액_표준편차,매입액_표준편차,매출총이익_표준편차,부가가치세_표준편차,부가세차감전영업이익_표준편차,영업이익_표준편차,주말매출액_표준편차,사업장방문고객수_표준편차,사업장방문신규고객수_표준편차,배달매출액_표준편차,주말배달매출액_표준편차,카드매출액_표준편차,주말카드매출액_표준편차,상권_식별자
0,DMC(디지털미디어시티),2120098,카페,7.5727,5.2844,7.2851,4.7609,6.7029,10.8206,1.7354,18.2221,4.6328,0.7511,0.1341,7.6554,1.7541,DMC(디지털미디어시티)_카페
1,가락시장역,2120234,한식,66.7188,3.3482,63.9273,25.6600,62.2638,84.1411,17.0500,6.9125,4.1122,15.3527,5.3017,64.8751,16.1978,가락시장역_한식
2,가로수길,2120186,술집,24.6029,2.9186,24.4196,13.7904,21.0899,33.9835,11.4162,4.0024,3.7921,0.4714,0.1234,24.7073,11.4366,가로수길_술집
3,가로수길,2120186,양식,47.4696,5.5149,46.8361,26.5625,37.0288,62.5175,20.9340,14.4391,11.7155,0.3319,0.0989,47.5501,20.9460,가로수길_양식
4,가로수길,2120186,일식,43.2368,4.5679,45.0980,13.3462,43.9535,57.0685,7.4536,1.9332,1.7003,0.0008,0.0003,43.2368,7.4536,가로수길_일식
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,홍대입구역(홍대),2120103,술집,49.1173,4.4199,47.4422,25.1515,45.5739,70.3352,25.2303,11.1215,7.0121,8.4986,3.0639,49.2780,25.3462,홍대입구역(홍대)_술집
173,홍대입구역(홍대),2120103,양식,21.7228,4.6233,21.0248,15.6416,19.1330,34.4439,9.4146,5.9535,5.1722,6.3280,0.7660,21.9887,9.6435,홍대입구역(홍대)_양식
174,홍대입구역(홍대),2120103,일식,14.7539,2.3404,14.2982,7.1473,16.0170,21.8626,6.4650,5.9388,4.7855,1.1049,0.3700,14.1164,6.2266,홍대입구역(홍대)_일식
175,홍대입구역(홍대),2120103,카페,7.7766,2.5314,6.8361,3.6824,6.6363,7.0442,3.5475,8.6426,5.1141,3.3612,0.4447,7.2872,3.6716,홍대입구역(홍대)_카페


```
* standard_deviation_population(kcd.transactions.sum_sales_card + kcd.transactions.sum_sales_delivery)
* standard_deviation_population(kcd.transactions.sum_purchase_card + kcd.transactions.sum_purchase_cash)
* standard_deviation_population((kcd.transactions.sum_sales_card + kcd.transactions.sum_sales_delivery) - (kcd.transactions.sum_purchase_card + kcd.transactions.sum_purchase_cash))
* standard_deviation_population(kcd.transactions.sum_sales_invoice - kcd.transactions.sum_purchase_invoice)
* standard_deviation_population(((kcd.transactions.sum_sales_card + kcd.transactions.sum_sales_delivery) - (kcd.transactions.sum_purchase_card + kcd.transactions.sum_purchase_cash)) - kcd.transactions.monthly_rental_fee)
* standard_deviation_population((((kcd.transactions.sum_sales_card + kcd.transactions.sum_sales_delivery) - (kcd.transactions.sum_purchase_card + kcd.transactions.sum_purchase_cash)) - kcd.transactions.monthly_rental_fee) - (kcd.transactions.sum_sales_invoice - kcd.transactions.sum_purchase_invoice))
* standard_deviation_population(kcd.transactions.sum_weekend_sales_card + kcd.transactions.sum_weekend_sales_delivery)
* standard_deviation_population(sum_customer_cnt)
* standard_deviation_population(sum_new_customer_cnt)
* STANDARD_DEVIATION_POPULATION(kcd.transactions.sum_sales_delivery)
* STANDARD_DEVIATION_POPULATION(kcd.transactions.sum_weekend_sales_delivery)
* STANDARD_DEVIATION_POPULATION(kcd.transactions.sum_sales_card)
* STANDARD_DEVIATION_POPULATION(kcd.transactions.sum_weekend_sales_card)
```


---

#### [SOHO_BUSINESS_DATA_LIST] 소호 임대동향, 사업장 정보

In [197]:
SOHO_BUSINESS_DATA_LIST = Preprocess.mergeSohoCredit(
    date_time1 = 202201,
    date_time2 = 202301,
    file_path = "/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/SOHO_BUSINESS/SOHO_",
    file_name = "_BUSINESS_230911.csv",
    column_name_list = ["상권명", "외식업종_중분류",
                        "소호사업장_수", "경영위기사업장_총합", "프랜차이즈사업장_총합", "소상공인연령_평균", "소상공인업력_평균"],
    column_num = 2)

In [198]:
SOHO_BUSINESS_DATA_LIST[0].info()
SOHO_BUSINESS_DATA_LIST[14]

<class 'pandas.core.frame.DataFrame'>
Index: 172 entries, 0 to 171
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   상권명          172 non-null    object 
 1   외식업종_중분류     172 non-null    object 
 2   소호사업장_수      172 non-null    float64
 3   경영위기사업장_총합   172 non-null    float64
 4   프랜차이즈사업장_총합  172 non-null    float64
 5   소상공인연령_평균    172 non-null    float64
 6   소상공인업력_평균    172 non-null    float64
 7   상권_식별자       172 non-null    object 
dtypes: float64(5), object(3)
memory usage: 12.1+ KB


,상권명,외식업종_중분류,소호사업장_수,경영위기사업장_총합,프랜차이즈사업장_총합,소상공인연령_평균,소상공인업력_평균,상권_식별자
0,DMC(디지털미디어시티),카페,9.0000,4.0000,3.0000,40.1111,4.9556,DMC(디지털미디어시티)_카페
1,가락시장역,한식,7.0000,0.0000,3.0000,43.1143,4.5429,가락시장역_한식
2,가로수길,술집,6.0000,1.0000,2.0000,37.9167,5.0167,가로수길_술집
3,가로수길,양식,6.0000,2.0000,0.0000,45.7167,3.5667,가로수길_양식
4,가로수길,일식,6.0000,0.0000,1.0000,36.2333,3.7667,가로수길_일식
...,...,...,...,...,...,...,...,...
176,홍대입구역(홍대),술집,44.0000,4.0000,2.0000,39.4432,4.0000,홍대입구역(홍대)_술집
177,홍대입구역(홍대),양식,10.0000,1.0000,2.0000,37.6500,5.8100,홍대입구역(홍대)_양식
178,홍대입구역(홍대),일식,8.0000,1.0000,0.0000,34.9125,2.5375,홍대입구역(홍대)_일식
179,홍대입구역(홍대),카페,16.0000,2.0000,0.0000,37.3000,4.6063,홍대입구역(홍대)_카페


```
* COUNT(*)
* SUM(kcd.transactions.is_risky)
* SUM(kcd.meta-info.is_franchise)
* AVERAGE(kcd.meta-info.age)
* AVERAGE(kcd.meta-info.duration)
```

* ValueError: could not convert string to float: 'DMC(디지털미디어시티)'

---

#### [SOHO_METAINFO_DATA_LIST] 소호 개인사업자 업력, 연령, 경영위기 FLAG, 프랜차이즈(브랜드) TF


In [199]:
SOHO_METAINFO_DATA_LIST = Preprocess.mergeSohoCredit(
    date_time1 = 202201,
    date_time2 = 202301,
    file_path = "/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/SOHO_METAINFO/SOHO_",
    file_name = "_METAINFO_230911.csv",
    column_name_list = ["상권명", "외식업종_중분류",
                        "정규고용인원_평균", "사업장방문고객수_평균", "사업장방문신규고객수_평균", "사업장임대면적_표준편차", "월임대료_표준편차", "임대보증금_표준편차"],
    column_num = 2)

In [200]:
SOHO_METAINFO_DATA_LIST[0].info()
SOHO_METAINFO_DATA_LIST[14]

<class 'pandas.core.frame.DataFrame'>
Index: 170 entries, 0 to 169
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   상권명            170 non-null    object 
 1   외식업종_중분류       170 non-null    object 
 2   정규고용인원_평균      170 non-null    float64
 3   사업장방문고객수_평균    170 non-null    float64
 4   사업장방문신규고객수_평균  170 non-null    float64
 5   사업장임대면적_표준편차   170 non-null    float64
 6   월임대료_표준편차      170 non-null    float64
 7   임대보증금_표준편차     170 non-null    float64
 8   상권_식별자         170 non-null    object 
dtypes: float64(6), object(3)
memory usage: 13.3+ KB


,상권명,외식업종_중분류,정규고용인원_평균,사업장방문고객수_평균,사업장방문신규고객수_평균,사업장임대면적_표준편차,월임대료_표준편차,임대보증금_표준편차,상권_식별자
0,DMC(디지털미디어시티),카페,0.7033,33.0211,8.2956,21.0944,0.9271,1.5947,DMC(디지털미디어시티)_카페
1,가락시장역,한식,1.9414,7.8517,5.3167,43.2950,1.9838,2.9983,가락시장역_한식
2,가로수길,술집,7.0683,4.4533,4.1133,51.4981,3.5908,3.4034,가로수길_술집
3,가로수길,양식,5.3750,12.0350,9.9800,232.0394,10.4152,16.8893,가로수길_양식
4,가로수길,일식,5.3750,3.3500,2.8883,63.1231,1.3525,3.0231,가로수길_일식
...,...,...,...,...,...,...,...,...,...
172,홍대입구역(홍대),술집,0.9261,9.2343,6.3218,66.8261,2.2330,3.4858,홍대입구역(홍대)_술집
173,홍대입구역(홍대),양식,1.6330,8.1444,6.6267,82.3801,2.0107,3.9547,홍대입구역(홍대)_양식
174,홍대입구역(홍대),일식,1.6675,6.7357,5.7886,56.7378,3.0323,2.5610,홍대입구역(홍대)_일식
175,홍대입구역(홍대),카페,0.7244,12.5631,7.6494,78.9299,2.4548,4.3513,홍대입구역(홍대)_카페


```
* AVERAGE(kcd.transactions.regular_employees_count)
* AVERAGE(kcd.transactions.sum_customer_cnt)
* AVERAGE(kcd.transactions.sum_new_customer_cnt)
* STANDARD_DEVIATION_POPULATION(kcd.transactions.business_square_size)
* STANDARD_DEVIATION_POPULATION(kcd.transactions.monthly_rental_fee)
* STANDARD_DEVIATION_POPULATION(kcd.transactions.rental_deposit)


---

#### [SOHO_STANDARDIZATION_DATA_LIST] 변동계수(coefficient of variation, CV) 산출을 위한 평균 연산

In [201]:
SOHO_STANDARDIZATION_DATA_LIST = Preprocess.mergeSohoCredit(
    date_time1 = 202201,
    date_time2 = 202301,
    file_path = "/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/SOHO_STANDARDIZATION/SOHO_",
    file_name = "_STANDARDIZATION_230916.csv",
    column_name_list = ["상권명", "상권코드", "외식업종_중분류",
                        "매출액_평균", "매입액_평균", "매출총이익_평균", "부가가치세_평균", "부가세차감전영업이익_평균", "영업이익_평균",
                        "주말매출액_평균", "사업장방문고객수_평균", "사업장방문신규고객수_평균",
                        "배달매출액_평균", "주말배달매출액_평균", "카드매출액_평균", "주말카드매출액_평균",
                        "사업장임대면적_평균", "월임대료_평균", "임대보증금_평균"],
    column_num = 3)

In [202]:
SOHO_STANDARDIZATION_DATA_LIST[0].info()
SOHO_STANDARDIZATION_DATA_LIST[14]

<class 'pandas.core.frame.DataFrame'>
Index: 170 entries, 0 to 169
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   상권명            170 non-null    object 
 1   상권코드           170 non-null    object 
 2   외식업종_중분류       170 non-null    object 
 3   매출액_평균         170 non-null    float64
 4   매입액_평균         170 non-null    float64
 5   매출총이익_평균       170 non-null    float64
 6   부가가치세_평균       170 non-null    float64
 7   부가세차감전영업이익_평균  170 non-null    float64
 8   영업이익_평균        170 non-null    float64
 9   주말매출액_평균       170 non-null    float64
 10  사업장방문고객수_평균    170 non-null    float64
 11  사업장방문신규고객수_평균  170 non-null    float64
 12  배달매출액_평균       170 non-null    float64
 13  주말배달매출액_평균     170 non-null    float64
 14  카드매출액_평균       170 non-null    float64
 15  주말카드매출액_평균     170 non-null    float64
 16  사업장임대면적_평균     170 non-null    float64
 17  월임대료_평균        170 non-null    float64
 18  임대보증금_평균       

,상권명,상권코드,외식업종_중분류,매출액_평균,매입액_평균,매출총이익_평균,부가가치세_평균,부가세차감전영업이익_평균,영업이익_평균,주말매출액_평균,사업장방문고객수_평균,사업장방문신규고객수_평균,배달매출액_평균,주말배달매출액_평균,카드매출액_평균,주말카드매출액_평균,사업장임대면적_평균,월임대료_평균,임대보증금_평균,상권_식별자
0,DMC(디지털미디어시티),2120098,카페,18.5791,5.7862,12.7929,-9.7112,10.5596,20.2707,1.4904,33.0211,8.2956,0.3668,0.0604,18.2122,1.4301,37.7215,2.2333,3.8889,DMC(디지털미디어시티)_카페
1,가락시장역,2120234,한식,61.4920,4.6577,56.8343,-30.7198,53.1857,83.9055,14.7331,7.8517,5.3167,6.2677,2.1644,55.2243,12.5687,99.1514,3.6486,5.7857,가락시장역_한식
2,가로수길,2120186,술집,36.8047,5.9847,30.8200,-18.9460,27.0300,45.9760,13.0253,4.4533,4.1133,0.2108,0.0552,36.5939,12.9701,95.4600,3.7900,4.5000,가로수길_술집
3,가로수길,2120186,양식,48.9948,6.0306,42.9641,-27.2917,34.0858,61.3775,20.5610,12.0350,9.9800,0.2219,0.0567,48.7729,20.5043,219.7318,8.8783,12.5000,가로수길_양식
4,가로수길,2120186,일식,57.5946,6.8616,50.7330,-24.9314,47.0830,72.0144,12.4009,3.3500,2.8883,0.0000,0.0000,57.5946,12.4009,92.2783,3.6500,5.8333,가로수길_일식
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,홍대입구역(홍대),2120103,술집,38.5479,5.9182,32.6296,-16.5625,29.5410,46.1034,17.7155,9.2343,6.3218,1.3418,0.4838,37.2061,17.2318,108.8480,3.0887,4.4091,홍대입구역(홍대)_술집
173,홍대입구역(홍대),2120103,양식,32.8962,5.4434,27.4528,-18.6236,24.6328,43.2564,12.6594,8.1444,6.6267,3.7778,0.6264,29.1184,12.0330,82.7290,2.8200,3.9000,홍대입구역(홍대)_양식
174,홍대입구역(홍대),2120103,일식,23.4557,4.8809,18.5748,-10.2309,14.8311,25.0620,9.6211,6.7357,5.7886,0.5623,0.2054,22.8934,9.4157,92.6002,3.7437,4.3125,홍대입구역(홍대)_일식
175,홍대입구역(홍대),2120103,카페,15.0101,3.8741,11.1360,-5.6691,7.7979,13.4670,5.5855,12.5631,7.6494,1.0215,0.1608,13.9885,5.4247,107.0284,3.3381,5.0625,홍대입구역(홍대)_카페


```
 * AVERAGE(kcd.transactions.sum_sales_card + kcd.transactions.sum_sales_delivery)
 * AVERAGE(kcd.transactions.sum_purchase_card + kcd.transactions.sum_purchase_cash)
 * AVERAGE((kcd.transactions.sum_sales_card + kcd.transactions.sum_sales_delivery) - (kcd.transactions.sum_purchase_card + kcd.transactions.sum_purchase_cash))
 * AVERAGE(kcd.transactions.sum_sales_invoice - kcd.transactions.sum_purchase_invoice)
 * AVERAGE(((kcd.transactions.sum_sales_card + kcd.transactions.sum_sales_delivery) - (kcd.transactions.sum_purchase_card + kcd.transactions.sum_purchase_cash)) - kcd.transactions.monthly_rental_fee)
 * AVERAGE((((kcd.transactions.sum_sales_card + kcd.transactions.sum_sales_delivery) - (kcd.transactions.sum_purchase_card + kcd.transactions.sum_purchase_cash)) - kcd.transactions.monthly_rental_fee) - (kcd.transactions.sum_sales_invoice - kcd.transactions.sum_purchase_invoice))
 * AVERAGE(kcd.transactions.sum_weekend_sales_card + kcd.transactions.sum_weekend_sales_delivery)
 * AVERAGE(kcd.transactions.sum_customer_cnt)
 * AVERAGE(kcd.transactions.sum_new_customer_cnt)
 * AVERAGE(kcd.transactions.sum_sales_delivery)
 * AVERAGE(kcd.transactions.sum_weekend_sales_delivery)
 * AVERAGE(kcd.transactions.sum_sales_card)
 * AVERAGE(kcd.transactions.sum_weekend_sales_card)
 * AVERAGE(kcd.transactions.business_square_size)
 * AVERAGE(kcd.transactions.monthly_rental_fee)
 * AVERAGE(kcd.transactions.rental_deposit)
```

---

### 2.한국신용데이터(KCD)의 익명처리 완료한 월별-상권별-업종별 소호 데이터 병합

In [203]:
class JoinData():

  def __init__(self, SOHO_INCOME_DATA: List, SOHO_BUSINESS_DATA: List,
               SOHO_METAINFO_DATA: List, SOHO_STANDARDIZATION_DATA: List, SOHO_MERGE_DATA_LIST = None):
    self.SOHO_INCOME_DATA = SOHO_INCOME_DATA
    self.SOHO_BUSINESS_DATA = SOHO_BUSINESS_DATA
    self.SOHO_METAINFO_DATA = SOHO_METAINFO_DATA
    self.SOHO_STANDARDIZATION_DATA = SOHO_STANDARDIZATION_DATA
    self.SOHO_MERGE_DATA_LIST = SOHO_MERGE_DATA_LIST

  def mergeDataset(self):
    # SOHO_DATA_COMBINATION = list(combinations(SOHO_DATA_LIST, 2))
      # 리스트에 있는 데이터의 모든 조합 구하기
      # <bound method JoinData.mergeDataset of <__main__.JoinData object at 0x7dbc92fcaef0>> : 어떤 클래스에 속해 있는 method

    def dropColumn(column_상권명, column_외식업종_중분류, dataframe):
      dataframe.drop([column_상권명, column_외식업종_중분류], axis = 1, inplace = True)

    for index in range(0, 15, 1):
      dropColumn("상권명", "외식업종_중분류", self.SOHO_INCOME_DATA[index])
      dropColumn("상권명", "외식업종_중분류", self.SOHO_BUSINESS_DATA[index])
      dropColumn("상권명", "외식업종_중분류", self.SOHO_METAINFO_DATA[index])
      dropColumn("상권명", "외식업종_중분류", self.SOHO_STANDARDIZATION_DATA[index])

    SOHO_MERGE_DATA_LIST: List = []
    for index in range(0, 15, 1):
      # [0 ~ 11] 2022.01 ~ 2022.12
      # [12 ~ 14] 2023.01 ~ 2023.03
      LEFT_DATA = pd.merge(left = self.SOHO_INCOME_DATA[index], right = self.SOHO_BUSINESS_DATA[index], on = "상권_식별자", how = "outer")
      LEFT_DATA = pd.merge(left = LEFT_DATA, right = self.SOHO_METAINFO_DATA[index], on = "상권_식별자", how = "outer")
      LEFT_DATA = pd.merge(left = LEFT_DATA, right = self.SOHO_STANDARDIZATION_DATA[index], on = "상권_식별자", how = "outer")

      # display(LEFT_DATA.info())
      LEFT_DATA.drop(["상권코드_y", "사업장방문고객수_평균_y", "사업장방문신규고객수_평균_y"], axis = 1, inplace = True)
      SOHO_MERGE_DATA_LIST.append(LEFT_DATA)
      LEFT_DATA = 0

    return SOHO_MERGE_DATA_LIST


In [204]:
CrossJoin = JoinData(SOHO_INCOME_DATA_LIST, SOHO_BUSINESS_DATA_LIST, SOHO_METAINFO_DATA_LIST, SOHO_STANDARDIZATION_DATA_LIST)

In [ ]:
SOHO_MERGE_DATA_LIST = CrossJoin.mergeDataset()

In [ ]:
SOHO_MERGE_DATA_LIST[0].info()
pd.set_option("display.max_column", None)
SOHO_MERGE_DATA_LIST[14]
  # JOIN ERROR: DUPLICATE ROW

In [210]:
pd.concat(SOHO_MERGE_DATA_LIST)

,상권코드_x,매출액_표준편차,매입액_표준편차,매출총이익_표준편차,부가가치세_표준편차,부가세차감전영업이익_표준편차,영업이익_표준편차,주말매출액_표준편차,사업장방문고객수_표준편차,사업장방문신규고객수_표준편차,배달매출액_표준편차,주말배달매출액_표준편차,카드매출액_표준편차,주말카드매출액_표준편차,상권_식별자,소호사업장_수,경영위기사업장_총합,프랜차이즈사업장_총합,소상공인연령_평균,소상공인업력_평균,정규고용인원_평균,사업장방문고객수_평균_x,사업장방문신규고객수_평균_x,사업장임대면적_표준편차,월임대료_표준편차,임대보증금_표준편차,매출액_평균,매입액_평균,매출총이익_평균,부가가치세_평균,부가세차감전영업이익_평균,영업이익_평균,주말매출액_평균,배달매출액_평균,주말배달매출액_평균,카드매출액_평균,주말카드매출액_평균,사업장임대면적_평균,월임대료_평균,임대보증금_평균
0,2120098,7.6766,9.8450,9.2697,6.1576,9.1012,12.2121,1.7788,20.5596,3.3266,1.3972,0.4727,7.1460,1.6387,DMC(디지털미디어시티)_카페,13.0000,0.0000,3.0000,39.5846,5.2692,0.8292,29.1015,6.6692,14.0824,1.0078,2.3026,15.3962,5.7894,9.6068,-8.8020,7.7199,16.5218,1.4247,0.6481,0.1842,14.7481,1.2404,34.1172,1.8869,4.0769
1,2120234,42.4509,2.4112,40.1029,14.5653,38.4800,52.3236,13.6707,5.1309,3.3644,13.0019,3.8753,40.6344,13.1709,가락시장역_한식,5.0000,0.0000,3.0000,44.9400,5.8400,2.3500,4.9360,3.3360,50.2651,2.2517,3.4986,41.3209,4.1791,37.1418,-18.8620,33.1538,52.0159,11.3322,7.0888,2.0076,34.2321,9.3246,95.7500,3.9880,6.1000
2,2120186,25.2326,2.7855,22.7759,16.4136,18.8498,34.8480,9.1549,6.0430,5.4239,0.1032,0.0003,25.2479,9.1549,가로수길_술집,7.0000,0.0000,1.0000,42.3571,5.9000,5.2500,3.3886,2.9971,59.3748,4.1031,6.6701,17.4297,3.1389,14.2908,-12.5042,10.3037,22.8079,5.6699,0.0421,-0.0000,17.3875,5.6699,111.4814,3.9871,5.7143
3,2120186,25.1656,4.1835,21.5091,14.7552,20.4437,34.7004,6.2320,5.9381,5.3513,5.4961,1.7954,24.5104,5.7937,가로수길_일식,7.0000,0.0000,1.0000,34.6571,3.5714,4.3857,6.6600,5.9857,60.2966,1.3120,2.7572,51.3019,6.4147,44.8872,-22.2104,41.2729,63.4833,15.6342,2.2438,0.7330,49.0582,14.9012,95.7729,3.6143,6.0714
4,2120186,32.0864,5.1861,28.5434,13.9835,27.9569,41.7216,14.6776,7.8539,4.9931,NaN,0.0001,32.0864,14.6776,가로수길_한식,7.0000,0.0000,2.0000,41.9143,4.6429,3.9514,9.8900,6.4200,44.4870,1.0782,1.8295,53.2927,8.3375,44.9552,-23.5541,40.5745,64.1286,14.3511,-0.0000,0.0000,53.2927,14.3511,128.8971,4.3807,5.7143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,2120103,29.8897,6.0718,29.3173,14.4546,28.3046,41.1918,10.5002,5.4923,4.3575,7.5128,1.9820,29.8433,10.6854,홍대입구역(홍대)_한식,28.0000,3.0000,2.0000,37.7571,3.9571,1.6514,7.5168,5.7888,64.8593,1.8380,3.0030,32.2874,5.5347,26.7526,-15.2323,23.8651,39.0974,10.9508,3.6203,1.0476,28.6671,9.9032,72.9395,2.8875,4.0000
177,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,선릉역_카페,6.0000,1.0000,1.0000,35.3167,4.2167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"신촌역(신촌역, 신촌로터리)_카페",5.0000,0.0000,1.0000,42.8600,5.2400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"총신대입구역(이수, 총신대)_카페",5.0000,2.0000,2.0000,32.8800,3.9800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [214]:
pd.concat(SOHO_MERGE_DATA_LIST).to_csv("/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/SOHO_MERGE_DATA_LIST.csv", index = True)

### 3.오아시스비즈니스(부동산 플랫폼 기업)의 서울특별시 고유필지단위별 소상공인의 매출등급 데이터에 상권코드, 법정동읍면동코드, 행정동코드 등 다른 테이블과의 병합(JOIN) 키로 활용될 컬럼 병합

* soho_sales_grade_list

  2022년 11개월의 서울특별시 고유필지단위별 소상공인의 매출등급 데이터를 담은 리스트 반환

* self.soho_commerce_data

  2022년 12개월의 소호 상권별 신용거래정보 데이터프레임을 담은 리스트와 2022년 11개월의 서울특별시 고유필지단위별 소상공인의 매출등급 데이터를 담은 리스트에서 원소(월별 데이터프레임)를 하나씩 꺼내어 병합하는 작업 11회 반복

In [207]:
class LoadData():

  def __init__(self,
               commerce_district_data_path: str, commerce_district_data = None, soho_commerce_data = None):
    # 생성자를 활용하여 초기화하고 클래스 내부에 캡슐화

    commerce_district_data_chunk = pd.read_csv(
        commerce_district_data_path, chunksize = 10**5,
        index_col = False, na_values = "NaN", encoding = "utf-8")
    commerce_district_data_list = list(commerce_district_data_chunk)
      # 용량이 큰 csv 파일 읽어오기(fopen - fread와 유사한 방식)
    self.commerce_district_data = pd.concat(commerce_district_data_list)
    self.soho_commerce_data = soho_commerce_data


  def chunkDataMerge(self, column_name_list: List, codebook_filepath: str, district_file_path: str,
                     left_join_key1: str, right_join_key1: str, left_join_key2: str, right_join_key2: str):
    self.commerce_district_data.columns = column_name_list
    codebook = pd.read_csv(codebook_filepath, index_col = False)
    district_code = pd.read_csv(district_file_path, index_col = False, encoding = "euc-kr")

    self.commerce_district_data = pd.merge(
        left = self.commerce_district_data, right = codebook,
        left_on = left_join_key1, right_on = right_join_key1, how = "left")
    # display(self.commerce_district_data.info())
    # display(district_code.info())

    self.commerce_district_data.drop(right_join_key1, axis = 1, inplace = True)
    self.commerce_district_data = pd.merge(
        left = self.commerce_district_data, right = district_code[["상권_구분_코드_명", "상권_코드", "상권_코드_명"]],
        left_on = left_join_key2, right_on = right_join_key2, how = "left")

    return self.commerce_district_data


  def splitMonthlyData(self, month: int):
    soho_sales_grade_list: List = []
    while month < 202213:
      soho_sales_grade_list.append(
          self.commerce_district_data.loc[self.commerce_district_data["데이터기준연월"] == month])
      month += 1

    return soho_sales_grade_list


  def mergeData(self, soho_credit_list: List, sales_grade_list: List,
                left_join_key: str, right_join_key: str):
    self.soho_commerce_data: List = []
    for index in range(0, 11, 1):
      self.soho_commerce_data(
          pd.merge(left = soho_credit_list[index], right = sales_grade_list[index],
                  left_on = left_join_key, right_on = right_join_key, how = "left"))

    return self.soho_commerce_data

In [208]:
soho_sales_grade = LoadData(
    commerce_district_data_path = "/content/drive/MyDrive/2023BigContest/data/대회제공/필지단위 소상공인 매출등급 정보.csv")
commerce_district_sales = soho_sales_grade.chunkDataMerge(
    column_name_list = ["데이터기준연월", "필지고유번호", "법정동읍면동코드", "업종코드", "매출등급"],
    codebook_filepath = "/content/drive/MyDrive/2023BigContest/data/market_loc_map.csv",
    left_join_key1 = "법정동읍면동코드", right_join_key1 = "STDG_EMD_CD",
    district_file_path = "/content/drive/MyDrive/2023BigContest/data/서울시 상권분석서비스(상권영역).csv",
    left_join_key2 = "상권_코드", right_join_key2 = "상권_코드")

FileNotFoundError: ignored

In [ ]:
soho_sales_grade_list = soho_sales_grade.splitMonthlyData(month = 202201)

```
STDG_EMD_CD_codebook = pd.read_excel(
    "/content/drive/MyDrive/2023BigContest/data/대회제공/소상공인 매출등급 예상 대상 필지.xlsx",
    sheet_name = 0,
    index_col = None)
STDG_EMD_CD_codebook.drop(["SLS_GRD_2301", "SLS_GRD_2302"], axis = 1, inplace = True)
  # [법정동읍면동코드] 시, 구 아래에 두며 그 아래에는 통과 반이 있는, 법률로 지정한 행정 구역의 정보를 나타내는 기호 체계
  # 법정동(10자리)의 구성: 광역시(2자리)+시군구(3자리)+읍면동(3자리)+리(2자리)
  # 법정동코드 10자리 중 앞6번째부터 8번째자리(101:청운동, 101:신교동…)
display(STDG_EMD_CD_codebook)
  # 고유식별자는 총 10,000행
  # [매출등급] 특정 필지의 '평당추정매출/평당월임대료'
```

In [ ]:
display(credit_data_list[10]), display(soho_sales_grade_list[10])
  # 2022년과 2023의 상권코드는 체계가 다른것으로 추정되어 TABLE JOIN하였으나 모두 NULL값으로 반환

In [ ]:
soho_commerce_merge_data = soho_sales_grade.mergeData(
    soho_credit_list = credit_data_list, sales_grade_list = soho_sales_grade_list,
    left_join_key = "상권코드", right_join_key = "상권_코드")

```
# VarianceInflationFactor()
# VIF(분산팽창인자) 값에 따른 최소한 모형 적합 시 입력할 독립변수의 수 파악을 위한 반복문 수행
  def VarainceInflationFactor(self,
                              X_train_data: pd.DataFrame(), X_test_data: pd.DataFrame(),
                              # num_variables: int,
                              Y_train_data: pd.DataFrame(), Y_test_data: pd.DataFrame()):

    evaluation_train = pd.DataFrame()
    evaluation_test = pd.DataFrame()

    for index in tqdm( range(1, len(X_train_data.columns) + 1)):

    # 분산팽창인자를 피처 수별로 구한 후 오름차순하여 vif 데이터프레임에 저장(다중공선성 문제로 나머지 변수 제외)
    # X_train_data에서 변수 index 수만큼의 피처(독립변수)만 남기고 나머지 변수 제외하여 X_train_fe_scaling_multiple 생성
    # X_train_test도 이하 상동
      vif = pd.DataFrame()
      vif["VIF_factor"] = [variance_inflation_factor(X_train_data.values, index)
                          for index in range(X_train_data.shape[1]) ]
      vif["Features"] = X_train_data.columns
      # print(vif)
      X_column_vif = vif.sort_values(by = "VIF_factor", ascending = True)["Features"][:index].values
        # TypeError: 'NoneType' object is not subscriptable

      X_train_fe_scaling_multiple, X_test_fe_scaling_multiple = X_train_data[X_column_vif].copy(), X_test_data[X_column_vif].copy()
```